In [519]:
import pandas as pd
import numpy as np

# import du csv
data = pd.read_csv('csv_correction.csv')
#data.info()

In [520]:
from sklearn import preprocessing, feature_selection, model_selection
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import cross_val_predict, cross_val_score
import time


In [521]:
data['fk_arb_etat_bool'] = data['fk_arb_etat'].replace(['ABATTU', 'EN PLACE', 'Non essouché', 'REMPLACÉ','SUPPRIMÉ'],'non essouché')

In [522]:
data

,longitude,latitude,clc_quartier,clc_secteur,haut_tot,haut_tronc,tronc_diam,fk_arb_etat,fk_stadedev,fk_port,...,fk_situation,fk_revetement,age_estim,fk_prec_estim,clc_nbr_diag,fk_nomtech,villeca,feuillage,remarquable,fk_arb_etat_bool
0,3.293264,49.840500,Quartier du Centre-Ville,Quai Gayant,6.0,2.0,37.0,EN PLACE,Jeune,semi libre,...,Alignement,Non,15.0,5.0,0.0,QUERUB,VILLE,Feuillu,Non,non essouché
1,3.273380,49.861409,Quartier du Vermandois,Stade Cepy,13.0,1.0,160.0,EN PLACE,Adulte,semi libre,...,Groupe,Non,50.0,10.0,0.0,PINNIGnig,VILLE,Conifère,Non,non essouché
2,3.289068,49.844513,Quartier du Centre-Ville,Rue Villebois Mareuil,12.0,3.0,116.0,REMPLACÉ,Adulte,semi libre,...,Alignement,Non,30.0,10.0,0.0,ACEPSE,VILLE,Feuillu,Non,non essouché
3,3.302387,49.861778,Quartier de l'Europe,Square des Marronniers,16.0,3.0,150.0,EN PLACE,Adulte,semi libre,...,Groupe,Non,50.0,2.0,0.0,ACEPLA,VILLE,Feuillu,Non,non essouché
4,3.304047,49.858446,Quartier de l'Europe,Avenue Buffon,5.0,2.0,170.0,Essouché,Adulte,réduit,...,Isolé,Non,40.0,2.0,0.0,SALBAB,VILLE,Feuillu,Non,Essouché
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7404,3.283505,49.848695,Quartier du Centre-Ville,Place Edouard Branly,3.0,2.0,15.0,EN PLACE,Jeune,Libre,...,Isolé,Non,5.0,1.0,0.0,ALBJUL,VILLE,Feuillu,Non,non essouché
7405,3.283436,49.848682,Quartier du Centre-Ville,Place Edouard Branly,3.0,2.0,15.0,EN PLACE,Jeune,Libre,...,Isolé,Non,3.0,1.0,0.0,ALBJULcho,VILLE,Feuillu,Non,non essouché
7406,3.297078,49.860413,Quartier de l'Europe,rue Hertz,3.0,2.0,15.0,EN PLACE,Jeune,Libre,...,Isolé,Non,5.0,1.0,0.0,CEDLIB,VILLE,Conifère,Non,non essouché
7407,3.291685,49.848009,Quartier du Centre-Ville,Square rue Fréreuse,3.0,2.0,15.0,EN PLACE,Jeune,Libre,...,Alignement,Non,5.0,1.0,0.0,FAGSYLdawgol,VILLE,Feuillu,Non,non essouché


## Grid Search

In [523]:
list_col = ['fk_stadedev', 'fk_arb_etat', 'fk_arb_etat_bool', 'clc_quartier', 'clc_secteur', 'feuillage', 'fk_port', 'fk_situation', 'fk_nomtech']
for col in list_col:
  encoder = preprocessing.OrdinalEncoder()
  fk_stadedev_encoded = encoder.fit_transform(data[[col]])
  print(encoder.categories_)
  data[col] = fk_stadedev_encoded

[array(['Adulte', 'Jeune', 'senescent', 'vieux'], dtype=object)]
[array(['ABATTU', 'EN PLACE', 'Essouché', 'Non essouché', 'REMPLACÉ',
       'SUPPRIMÉ'], dtype=object)]
[array(['Essouché', 'non essouché'], dtype=object)]
[array(['HARLY', 'OMISSY', 'Quartier Remicourt', 'Quartier Saint-Jean',
       'Quartier Saint-Martin - Oëstres', 'Quartier de Neuville',
       "Quartier de l'Europe", 'Quartier du Centre-Ville',
       'Quartier du Vermandois', "Quartier du faubourg d'Isle", 'ROUVROY'],
      dtype=object)]
[array(['Accueil de loisirs Kergomard', 'Ancienne ecole David et Maigret',
       'Auberge de jeunesse', 'Avenue Aristide Briand', 'Avenue Buffon',
       'Avenue Charles Feuillette', 'Avenue Faidherbe',
       'Avenue Léo Lagrange', 'Avenue Pierre Chocquart',
       'Avenue Pierre Choquart', 'Avenue Robert Schuman',
       'Avenue de la Paix', 'Avenue de la République',
       'Avenue de la Résistance',
       'Avenue des Fusillés de Fontaine Notre Dame',
       'Avenue du Cimet

Split des données

In [524]:
all_cols_list = ["haut_tot", 'haut_tronc', 'tronc_diam', 'fk_stadedev', 'age_estim', 'clc_quartier', 'clc_secteur', 'feuillage', 'fk_port', 'fk_situation', 'fk_nomtech']

In [525]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(data[all_cols_list], data["fk_arb_etat_bool"], test_size=0.2, random_state=42)

In [526]:
# train
classifier = RandomForestClassifier(random_state=42, n_jobs=-1)
params = {"n_estimators": [50, 100, 150, 200, 250, 300]}
gscv = GridSearchCV(estimator=classifier, param_grid=params, scoring="accuracy", cv=5, n_jobs=-1)
classifier_with_grid_search = gscv.fit(X_train, y_train)  # le meilleur modele

print(classifier_with_grid_search.best_params_)
from sklearn.metrics import accuracy_score
cvs = cross_val_predict(gscv, X_train, y_train, cv = 3, n_jobs=-1)

best_n = classifier_with_grid_search.best_params_["n_estimators"]
best_n




{'n_estimators': 250}


250

In [527]:
confusion_matrix(y_train, cvs, normalize="true")

array([[0.15      , 0.85      ],
       [0.00345602, 0.99654398]])

On constate que la matrice de confusion est pas bonne, pas du tout satisfaisante

In [528]:
# TEST (meme si pas besoion car modele pas validé)
# y_test_predicted = classifier_with_grid_search.predict(X_test)
# accuracy_score(y_test, y_test_predicted)

Feature Selection

In [529]:
all_cols_list = ["haut_tot", 'haut_tronc', 'tronc_diam', 'fk_stadedev', 'age_estim', 'clc_quartier', 'clc_secteur', 'feuillage', 'fk_port', 'fk_situation', 'fk_nomtech']

In [530]:
list_col = ['fk_stadedev', 'fk_arb_etat', 'fk_arb_etat_bool', 'clc_quartier', 'clc_secteur', 'feuillage', 'fk_port', 'fk_situation', 'fk_nomtech']
for col in list_col:
  encoder = preprocessing.OrdinalEncoder()
  fk_stadedev_encoded = encoder.fit_transform(data[[col]])
  #print(encoder.categories_)
  data[col] = fk_stadedev_encoded

In [531]:
# Train the Random Forest model
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)

# Evaluate the model
accuracy_before = rf.score(X_test, y_test)

In [532]:
accuracy_before

0.979757085020243

In [533]:
importances = rf.feature_importances_
feature_names = data[all_cols_list].columns

feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})
feature_importance_df

,Feature,Importance
0,haut_tot,0.151611
1,haut_tronc,0.112041
2,tronc_diam,0.268431
3,fk_stadedev,0.013490
4,age_estim,0.073595
5,clc_quartier,0.060569
6,clc_secteur,0.129609
7,feuillage,0.011906
8,fk_port,0.024605
9,fk_situation,0.030371


In [534]:
# Rank features by importance
feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
print(feature_importance_df)

         Feature  Importance
2     tronc_diam    0.268431
0       haut_tot    0.151611
6    clc_secteur    0.129609
10    fk_nomtech    0.123771
1     haut_tronc    0.112041
4      age_estim    0.073595
5   clc_quartier    0.060569
9   fk_situation    0.030371
8        fk_port    0.024605
3    fk_stadedev    0.013490
7      feuillage    0.011906


In [535]:
# Select top N features (example selecting top 5 features)
top_features = feature_importance_df['Feature'][:5].values
X_train_selected = X_train[top_features]
X_test_selected = X_test[top_features]

In [536]:
# Train the Random Forest model with selected features
start_time = time.time()
rf_selected = RandomForestClassifier(n_estimators=best_n, random_state=42, n_jobs=-1)
rf_selected.fit(X_train_selected, y_train)
end_time = time.time()
y_test_predicted = rf_selected.predict(X_test_selected)


In [537]:
# validation
from sklearn.metrics import accuracy_score
cvs = cross_val_predict(gscv, X_train, y_train, cv = 3, n_jobs=-1)
confusion_matrix(y_train, cvs, normalize="true")


array([[0.15      , 0.85      ],
       [0.00345602, 0.99654398]])

In [538]:
# TEST
#accuracy_after = rf_selected.score(X_test_selected, y_test)
print(f'Accuracy after feature selection: {accuracy_score(y_test, y_test_predicted):.3f}')

print(f'Temps écoulé : {end_time - start_time :.3f} secondes.')

Accuracy after feature selection: 0.980
Temps écoulé : 0.396 secondes.


Résultats de matrice de confusion très mauvais, trop grand déséquilibre entre les essouchés et les non essouchés

# SMOTE

On va utiliser SMOTE pour ajouter artificiellement des essouchés

In [539]:
from imblearn.over_sampling import SMOTE

In [540]:
data

,longitude,latitude,clc_quartier,clc_secteur,haut_tot,haut_tronc,tronc_diam,fk_arb_etat,fk_stadedev,fk_port,...,fk_situation,fk_revetement,age_estim,fk_prec_estim,clc_nbr_diag,fk_nomtech,villeca,feuillage,remarquable,fk_arb_etat_bool
0,3.293264,49.840500,7.0,112.0,6.0,2.0,37.0,1.0,1.0,8.0,...,0.0,Non,15.0,5.0,0.0,185.0,VILLE,1.0,Non,1.0
1,3.273380,49.861409,8.0,228.0,13.0,1.0,160.0,1.0,0.0,8.0,...,1.0,Non,50.0,10.0,0.0,129.0,VILLE,0.0,Non,1.0
2,3.289068,49.844513,7.0,166.0,12.0,3.0,116.0,4.0,0.0,8.0,...,0.0,Non,30.0,10.0,0.0,20.0,VILLE,1.0,Non,1.0
3,3.302387,49.861778,6.0,225.0,16.0,3.0,150.0,1.0,0.0,8.0,...,1.0,Non,50.0,2.0,0.0,15.0,VILLE,1.0,Non,1.0
4,3.304047,49.858446,6.0,4.0,5.0,2.0,170.0,2.0,0.0,6.0,...,2.0,Non,40.0,2.0,0.0,192.0,VILLE,1.0,Non,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7404,3.283505,49.848695,7.0,103.0,3.0,2.0,15.0,1.0,1.0,1.0,...,2.0,Non,5.0,1.0,0.0,30.0,VILLE,1.0,Non,1.0
7405,3.283436,49.848682,7.0,103.0,3.0,2.0,15.0,1.0,1.0,1.0,...,2.0,Non,3.0,1.0,0.0,31.0,VILLE,1.0,Non,1.0
7406,3.297078,49.860413,6.0,238.0,3.0,2.0,15.0,1.0,1.0,1.0,...,2.0,Non,5.0,1.0,0.0,50.0,VILLE,0.0,Non,1.0
7407,3.291685,49.848009,7.0,226.0,3.0,2.0,15.0,1.0,1.0,1.0,...,0.0,Non,5.0,1.0,0.0,70.0,VILLE,1.0,Non,1.0


Split des données en train et test

In [541]:
sm = SMOTE(random_state=42)
X_smoted, y_smoted = sm.fit_resample(X_train_selected, y_train)

In [542]:
X_smoted

,tronc_diam,haut_tot,clc_secteur,fk_nomtech,haut_tronc
0,110.000000,13.000000,90.000000,20.000000,2.000000
1,115.000000,13.000000,34.000000,94.000000,2.000000
2,113.000000,16.000000,82.000000,132.000000,5.000000
3,165.000000,14.000000,10.000000,45.000000,2.000000
4,80.000000,8.000000,14.000000,169.000000,3.000000
...,...,...,...,...,...
11569,212.491436,25.000000,91.000000,17.491436,14.498287
11570,139.231563,15.725313,209.342813,23.240625,2.175938
11571,55.243118,8.054063,59.810724,142.621559,0.702709
11572,110.823308,17.050877,141.164662,129.000000,4.721554


In [543]:
y_smoted

0        1.0
1        1.0
2        1.0
3        1.0
4        1.0
        ... 
11569    0.0
11570    0.0
11571    0.0
11572    0.0
11573    0.0
Name: fk_arb_etat_bool, Length: 11574, dtype: float64

In [544]:
classifier = RandomForestClassifier(random_state=42, n_jobs=-1, n_estimators=best_n)
classifier_with_grid_search_and_FS_and_SMOTE = classifier.fit(X_smoted, y_smoted)  # le meilleur modele

Validation

In [545]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_predict, cross_val_score

In [546]:
cvs = cross_val_predict(gscv, X_smoted, y_smoted, cv = 3, n_jobs=-1)
confusion_matrix(y_smoted, cvs, normalize="true")

array([[0.97909107, 0.02090893],
       [0.01676171, 0.98323829]])

In [547]:
np.mean(cross_val_score(classifier, X_smoted, y_smoted, cv = 3, n_jobs=-1))

0.981078278900985

On peut rajouter 

La matrice de confusion est satisfaisante, on passe aux tests.

In [548]:
from sklearn.metrics import accuracy_score

y_test_predicted = classifier_with_grid_search_and_FS_and_SMOTE.predict(X_test_selected)
accuracy_score(y_test, y_test_predicted)

0.9723346828609987

In [549]:
y_pred = classifier_with_grid_search_and_FS_and_SMOTE.predict(X_test_selected)
confusion_matrix(y_test, y_pred, normalize="true")

array([[0.2       , 0.8       ],
       [0.01441318, 0.98558682]])

In [550]:
y_pred

array([1., 1., 1., ..., 1., 1., 1.])

In [551]:
data.columns

Index(['longitude', 'latitude', 'clc_quartier', 'clc_secteur', 'haut_tot',
       'haut_tronc', 'tronc_diam', 'fk_arb_etat', 'fk_stadedev', 'fk_port',
       'fk_pied', 'fk_situation', 'fk_revetement', 'age_estim',
       'fk_prec_estim', 'clc_nbr_diag', 'fk_nomtech', 'villeca', 'feuillage',
       'remarquable', 'fk_arb_etat_bool'],
      dtype='object')